In [19]:
import stanza
import graphviz
import os
import json

import networkx as nx
import pandas as pd
import numpy as np

from tuw_nlp.grammar.text_to_4lang import TextTo4lang
from tuw_nlp.graph.fourlang import FourLang
from tuw_nlp.text.pipeline import CachedStanzaPipeline, CustomStanzaPipeline
from graphviz import Source

In [20]:
config = {
    "lang": "en",
    "depth": 0,
    "substitute": False,
    "nr of samples": 100,
}
data = {
 "dir": "data"
}

In [3]:
text_to_4lang = TextTo4lang(config["lang"], config["lang"] + "_nlp_cache")

nlp = stanza.Pipeline(config["lang"])

2021-11-08 15:06:09 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-11-08 15:06:09 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2021-11-08 15:06:09 INFO: Use device: gpu
2021-11-08 15:06:09 INFO: Loading: tokenize
2021-11-08 15:06:10 INFO: Loading: pos
2021-11-08 15:06:11 INFO: Loading: lemma
2021-11-08 15:06:11 INFO: Loading: depparse
2021-11-08 15:06:11 INFO: Done loading processors!
2021-11-08 15:06:24 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| 

In [21]:
GOLD_ATTRIBUTES = "gold_attributes"
GOLD = "labels_gold"
ANNOTATED_ATTRIBUTES = "annotated_attributes"

def create_input(directory, attribute):
    sentence = []
    label = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), "rt") as f:
            doc = json.load(f)
            for sen in doc["sens"].values():
                if doc[GOLD]:
                    lab = attribute in sen[GOLD_ATTRIBUTES]
                else:
                    lab = attribute in sen[ANNOTATED_ATTRIBUTES]
                sentence.append(sen["text"])
                label.append(lab)
    return pd.DataFrame(data=list(zip(sentence,label)),columns=["Text","Label"])



In [22]:
#Load in Data for specific feature
train_data = create_input(os.path.join(data["dir"], "train"), "Planzeichen")

#Save the labels for later. In this case its just 0,1 conversion for Planzeichen
labels = train_data.Label*1

train_data_true = train_data[train_data["Label"] == True]
train_data_false = train_data[train_data["Label"] == False]
train_data_true = train_data_true[:config["nr of samples"]]
train_data_false = train_data_false[:config["nr of samples"]]
train_data = train_data_true
train_data = train_data.append(train_data_false)
train_data

,Text,Label
10,Gemäß § 5 (4) der Bauordnung für Wien wird für...,True
12,Die mit Eklw/BB2 bezeichnete Fläche ist der Er...,True
13,Von der mit Eklw P BB3 bezeichneten Fläche dür...,True
31,Auf den mit G/BB1 bezeichneten Flächen ist die...,True
32,In den mit BB2 bezeichneten Bereichen sind die...,True
...,...,...
123,Der bisher gültige Flächenwidmungsplan und Beb...,False
124,Die roten Planzeichen gelten als neu festgesetzt.,False
125,Für die rechtliche Bedeutung der roten Planzei...,False
126,Für die Querschnitte der Verkehrsflächen gemäß...,False


In [23]:
def create_fl(sentence, depth=config["depth"], substitute=config["substitute"]):
    fl_graphs = list(text_to_4lang(sentence, depth=int(depth), substitute=substitute))
    g = fl_graphs[0]
    for n in fl_graphs[1:]:
        g = nx.compose(g, n)
    fl = FourLang(g, 0)
    if int(depth):
        text_to_4lang.expand(fl, depth=int(depth), substitute=substitute)
    # sen = nlp(sentence).sentences[0]
    return fl

In [24]:
def visualize(sentence):
    dot = graphviz.Digraph()
    dot.node("0", "ROOT", shape="box")
    for token in sentence.tokens:
        for word in token.words:
            dot.node(str(word.id), word.text)
            dot.edge(str(word.head), str(word.id),
                     label=word.deprel)
    return dot

In [25]:
train_data["4lang"] = None
train_data

,Text,Label,4lang
10,Gemäß § 5 (4) der Bauordnung für Wien wird für...,True,None
12,Die mit Eklw/BB2 bezeichnete Fläche ist der Er...,True,None
13,Von der mit Eklw P BB3 bezeichneten Fläche dür...,True,None
31,Auf den mit G/BB1 bezeichneten Flächen ist die...,True,None
32,In den mit BB2 bezeichneten Bereichen sind die...,True,None
...,...,...,...
123,Der bisher gültige Flächenwidmungsplan und Beb...,False,None
124,Die roten Planzeichen gelten als neu festgesetzt.,False,None
125,Für die rechtliche Bedeutung der roten Planzei...,False,None
126,Für die Querschnitte der Verkehrsflächen gemäß...,False,None


In [28]:
for index, row in train_data.iterrows():
    # print(f"{index}: {row['Text'][:30]}")
    if row["Text"] and row["Text"] != "":
        fl = create_fl(row["Text"])
    else:
        fl = None
    train_data.at[index, "4lang"] = fl

10: Gemäß § 5 (4) der Bauordnung f
12: Die mit Eklw/BB2 bezeichnete F
13: Von der mit Eklw P BB3 bezeich
31: Auf den mit G/BB1 bezeichneten
32: In den mit BB2 bezeichneten Be
33: In den mit P/BB3 bezeichneten 
34: In dem mit BB4 bezeichneten Be
37: Gemäß § 4 Abs. 3 der BO für Wi
54: Innerhalb der mit G bezeichnet
60: Auf den mit BB1 bezeichneten G
61: Auf den mit BB2 bezeichneten u
63: Auf der mit BB3 bezeichneten u
64: Auf den mit BB4 bezeichneten G
65: Auf den mit BB5 bezeichneten G
66: Auf den mit BB6 bezeichneten G
67: Auf den mit BB7 bezeichneten u
68: Auf den mit BB8 bezeichneten u
70: Die mit BB9 bezeichneten und a
82: Für die mit BB 6 bezeichnete F
88: Die Dächer der auf den mit der
100: Bestimmungen gemäß § 5 Abs. 4 
101: Auf den mit G BB 2 bezeichnete
102: Auf den mit Esp BB 3 bezeichne
103: Auf den mit BB 4 bezeichneten 
105: Auf den mit BB 5 bezeichneten 
113: Gemäß § 5 (4) der Bauordnung f
114: Auf der mit Esp BB18 bezeichne
116: Auf der mit Esp BB19 bezeichne
131: In dem 

In [29]:
train_data

,Text,Label,4lang
10,Gemäß § 5 (4) der Bauordnung für Wien wird für...,True,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
12,Die mit Eklw/BB2 bezeichnete Fläche ist der Er...,True,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
13,Von der mit Eklw P BB3 bezeichneten Fläche dür...,True,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
31,Auf den mit G/BB1 bezeichneten Flächen ist die...,True,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
32,In den mit BB2 bezeichneten Bereichen sind die...,True,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
...,...,...,...
123,Der bisher gültige Flächenwidmungsplan und Beb...,False,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
124,Die roten Planzeichen gelten als neu festgesetzt.,False,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
125,Für die rechtliche Bedeutung der roten Planzei...,False,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
126,Für die Querschnitte der Verkehrsflächen gemäß...,False,<tuw_nlp.graph.fourlang.FourLang object at 0x7...
